In [0]:
import dlt
from pyspark.sql.functions import *

# Schema da cui leggere i dati Bronze
bronze_schema = "catalog_progetto_finale.bronze_schema_pf"

# Lettura delle tabelle Bronze (Delta) e le carica come DataFrame Spark standard
#pulisco le tabelle dalla colonna _rescued_data perchè non serve più e può creare problemi nelle fasi successive
hotels_bronze_df = spark.read.table(f"{bronze_schema}.hotels_bronze").drop("_rescued_data")
customers_bronze_df = spark.read.table(f"{bronze_schema}.customers_bronze").drop("_rescued_data")
rooms_bronze_df = spark.read.table(f"{bronze_schema}.rooms_bronze").drop("_rescued_data")
bookings_bronze_df = spark.read.table(f"{bronze_schema}.bookings_bronze").drop("_rescued_data")
payments_bronze_df = spark.read.table(f"{bronze_schema}.payments_bronze").drop("_rescued_data")

@dlt.table(name="hotels_silver", comment="Hotel puliti, escludendo country='XX'.")
def hotels_silver():
    return hotels_bronze_df.filter(col("country") != "XX")

@dlt.table(name="customers_silver", comment="Clienti puliti, con email nulle gestite e duplicati rimossi.")
def customers_silver():
    return (
        customers_bronze_df
        .withColumn("email", when(trim(col("email")) == "", None).otherwise(col("email")))
        .dropDuplicates(["customer_id"])
    ) # trim rimuove gli spazi

@dlt.table(name="rooms_silver", comment="Stanze pulite, con duplicati rimossi.")
def rooms_silver():
    return rooms_bronze_df.dropDuplicates(["room_id"])

@dlt.table(name="bookings_silver", comment="Prenotazioni pulite con date corrette e valori validati.")
def bookings_silver():
    bookings_with_dates = bookings_bronze_df.withColumns({
        "checkin_date": to_timestamp(col("checkin_date")),
        "checkout_date": to_timestamp(col("checkout_date"))
    })
    bookings_fixed_dates = bookings_with_dates.withColumn(
        "temp_checkin", when(col("checkin_date") > col("checkout_date"), col("checkout_date")).otherwise(col("checkin_date"))
    ).withColumn(
        "checkout_date", when(col("checkin_date") > col("checkout_date"), col("checkin_date")).otherwise(col("checkout_date"))
    ).withColumn("checkin_date", col("temp_checkin")).drop("temp_checkin")
    
    valid_currencies = ['EUR', 'USD', 'GBP']
    return (
        bookings_fixed_dates
        .withColumn("total_amount", when(col("total_amount") < 0, None).otherwise(col("total_amount")))
        .withColumn("currency", when(col("currency").isin(valid_currencies), col("currency")).otherwise(None))
    )

# Per creare la tabella payments_silver, dobbiamo leggere la tabella bookings_silver appena definita.
# DLT ci permette di farlo usando dlt.read()
@dlt.table(name="payments_silver", comment="Pagamenti con flag di data quality.")
def payments_silver():
    bookings_silver_df = dlt.read("bookings_silver").select("booking_id", "total_amount")
    
    merged_df = payments_bronze_df.join(bookings_silver_df, "booking_id", "left")
    
    #dq_orphan -> colonna che vale true se il pagamento è "orfano", cioè se non ha trovato una prenotazione
    # dq_over_amount -> colonna che vale true se l'importo del pagamento (amount) è maggiore dell'importo totale della prenotazione
    return (
        merged_df
        .withColumn("dq_orphan", col("total_amount").isNull())
        .withColumn("dq_over_amount", col("amount") > col("total_amount"))
        .select(payments_bronze_df["*"], "dq_orphan", "dq_over_amount")
    )

Questo script orchestra una serie di pulizie e validazioni, trasformando dati grezzi e potenzialmente inaffidabili in un set di tabelle Silver pulite, coerenti e pronte per essere usate per analisi aggregate o per il machine learning.